In [ ]:
%pylab notebook 

In [ ]:
from pleque.io import _geqdsk as eqdsktool 
from pleque.io._readers import read_eqdsk
from pleque.utils.plotting import *
from pleque import Equilibrium
from test.testing_utils import get_test_equilibria_filenames, load_testing_equilibrium


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
gfiles = get_test_equilibria_filenames()
gfiles

In [ ]:
eq = load_testing_equilibrium()

In [ ]:
g = eq.grid(resolution=[1e-3, 1e-3])
g.psi.shape

In [ ]:
# first three files are fiesta and may cause some troubles
Npl = 4

eq = read_eqdsk(gfiles[Npl])

eq_iter = read_eqdsk(gfiles[4])
with open(gfiles[Npl], 'r') as f:
    eq_efit = eqdsktool.read(f)
eq_efit.keys()

plt.figure()
eq._plot_overview()

In [ ]:
plt.figure()
eq._plot_overview()
plot_extremes(eq)

#gfile

r_axis = np.linspace(eq_efit["rleft"], eq_efit["rleft"] + eq_efit["rdim"], eq_efit["nx"])
z_axis = np.linspace(eq_efit["zmid"] - eq_efit["zdim"] / 2, eq_efit["zmid"] + eq_efit["zdim"] / 2, eq_efit["ny"])

plt.figure()
ax = plt.gca()

ax.plot(eq_efit['rlim'], eq_efit['zlim'], color='k', lw=2)
ax.plot(eq_efit['rbdry'], eq_efit['zbdry'], 'C1--', lw=2)
ax.contour(r_axis, z_axis, eq_efit['psi'].T, 20)
ax.contour(r_axis, z_axis, eq_efit['psi'].T, [eq_efit['sibdry']])

ax.set_aspect('equal')



In [ ]:

plt.figure()
ax = plt.gca()
ax.plot(eq_efit['rlim'], eq_efit['zlim'], color='k', lw=2)
ax.plot(eq_efit['rbdry'], eq_efit['zbdry'], 'C0-.', lw=2)
ax.contour(r_axis, z_axis, eq_efit['psi'].T, [eq_efit['sibdry']], colors='C0')

eq.lcfs.plot(color='C1', ls='--')
ax.plot(eq._x_point[0], eq._x_point[1], 'C1+', lw=3)

ax.set_aspect('equal')
#ax.set_xlim((5.06, 5.09))
#ax.set_ylim((-3.45, -3.43))

In [ ]:
def plot_2d(R, Z, data, *args, title=None):
    
    rlim = [np.min(eq.first_wall.R), np.max(eq.first_wall.R)]
    zlim = [np.min(eq.first_wall.Z), np.max(eq.first_wall.Z)]

    size = rlim[1] - rlim[0]
    rlim[0] -= size / 12
    rlim[1] += size / 12

    size = zlim[1] - zlim[0]
    zlim[0] -= size / 12
    zlim[1] += size / 12
    
    plt.figure()    
    ax = plt.gca()
    
    ax.set_xlim(rlim)
    ax.set_ylim(zlim)
    
    ax.plot(eq.lcfs.R, eq.lcfs.Z, color='k', ls='--', lw=2)
    ax.plot(eq.first_wall.R, eq.first_wall.Z, 'k-', lw=2)
    ax.set_xlabel('R [m]')
    ax.set_ylabel('Z [m]')
    

    ax.set_aspect('equal')
    
    cl = ax.contour(R, Z, data, *args)
    
    if title is not None:
        ax.set_title(title)
        
    return cl

In [ ]:
grid = eq.grid((200,300),dim='size')

plot_2d(grid.R, grid.Z, grid.psi, 20, title=r'$\psi$')
plot_2d(grid.R, grid.Z, eq.B_pol(grid), 20, title=r'$B_\mathrm{p}$')
plot_2d(grid.R, grid.Z, eq.B_tor(grid), 20, title=r'$B_\mathrm{t}$')
plot_2d(grid.R, grid.Z, eq.B_abs(grid), 20, title=r'$|B|$')
cl = plot_2d(grid.R, grid.Z, eq.pressure(grid)/1e3, np.linspace(0, 8e5/1e3, 41), title=r'$p$ [kPa]')
plt.colorbar(cl)
cl = plot_2d(grid.R, grid.Z, eq.j_tor(grid), np.linspace(0, 1e7, 21), title=r'$j_\phi$')
plt.colorbar(cl)
cl = plot_2d(grid.R, grid.Z, eq.j_pol(grid), np.linspace(0, 3e5, 21), title=r'$j_\theta$')
cl = plot_2d(grid.R, grid.Z, eq.tor_flux(grid), np.linspace(-40, 0, 21), title='toroidal flux')
#plt.colorbar(cl)


## Profile mappings: 

In [ ]:
eq = read_eqdsk(gfiles[5])

In [ ]:
from scipy.special import erf


N = 200

chord1 = eq.coordinates(R=0.6*np.ones(N), Z=np.linspace(0.3, -0.1, N))
chord2 = eq.coordinates(R=np.linspace(0.35, 0.6, 20), Z=-0.1*np.ones(20))


# let's define some testing profile function: 
prof_func = lambda x, k1, xsep: k1/4 * (1 + erf((x-xsep)*20))*np.log((x+1)*1.2)
chord_prof = prof_func(1 - chord1.psi_n, 10, 0.15)

# todo here
plt.figure()
plt.plot(chord1.Z, chord_prof, color='C3')
eq.fluxfuncs.add_flux_func('test_prof', chord_prof, chord1, spline_smooth=0)
plt.plot(chord1.Z, eq.fluxfuncs.test_prof(chord1), '--', color='C3')
plt.xlabel('Z [m]')
plt.ylabel('profile value [a.u.]')


plt.figure()
plt.plot(chord2.R, eq.fluxfuncs.test_prof(chord2), '--', color='C4')
plt.xlabel('R [m]')
plt.ylabel('profile value [a.u.]')


plt.figure()
plt.plot(chord1.r_mid, chord_prof, color='C3')
plt.xlabel(r'$r_\mathrm{mid}$ [m]')
plt.ylabel('profile value [a.u.]')


plt.figure()
eq._plot_overview()

chord1.plot(lw=3, ls='--', color='C3')
chord2.plot(lw=3, ls='--', color='C4')
